# 背景建模

## 帧差法
由于场景中的目标在运动，目标的影响在不同的图像帧中的位置不同。该类算法对时间上连续的两帧图像进行查分运算，不同帧对应的像素点相减，判断灰度差的绝对值，当绝对值超过一定的阈值时，即可判断为运动的目标，从而实现目标的检测功能

## 混合高斯模型
在测试阶段，对新来的像素点值与混合高斯模型中的每一个均值进行比较，如果其差值在2倍的方差之间的话，则认为是背景，否则认为是前景。将前景赋值为255，背景赋值为0，这样就形成了一幅前景二值图

### 混合高斯模型学习方法
1、首先初始化每个高斯模型矩阵参数
2、取视频中T帧数据图像用来训练高斯混合模型，来了第一个像素之后用它来当做第一高斯分布
3、当后帧的像素值传来时，与前面已有的高斯的均值比较，如果该像素点的值与其模型均值差在3倍的方差内，则属于该分部，并对其进行参数更新
4、如果下一次来的像素不满足当前的高斯分布，用它来创建一个新的高斯分布

In [2]:
import numpy as np
import cv2

In [5]:
#测试视频
test_path = r'D:\\py_project\\test.avi'
cap = cv2.VideoCapture(test_path)
#形态学操作需要使用
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
#创建混合高斯模型用于背景建模
fgbg = cv2.createBackgroundSubtractorMOG2()

while(True):
    ret, frame = cap.read()
    fgmask = fgbg.apply(frame)
    #形态学开运算去噪点
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    #寻找视频中的轮廓
    im, contours, hierarchy = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for c in contours:
        #计算各轮廓的周长
        perimeter = cv2.arcLength(c,True)
        if perimeter > 188:
            #找到一个直矩形（不会旋转）
            x,y,w,h = cv2.boundingRect(c)
            #画出矩形
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)  

    cv2.imshow('frame', frame)
    cv2.imshow('fgmask', fgmask)
    k = cv2.waitKey(150) & 0xff
    if k == 27:
        break
            
cap.release()
cv2.destroyAllWindows()